<a href="https://colab.research.google.com/github/ad71/ragbot/blob/master/langgraph_agent_executor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
%pip install -qU langchain langchain_openai tavily-python langchainhub langgraph

In [9]:
import os
from google.colab import userdata


os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')

In [10]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

prompt = hub.pull('hwchase17/openai-functions-agent')
llm = ChatOpenAI(model='gpt-3.5-turbo-1106', streaming=True)
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

In [11]:
# define agent state

from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
    input: str # the input string
    chat_history: list[BaseMessage] # the list of previous messages in the conversation
    agent_outcome: Union[AgentAction, AgentFinish, None] # the outcome of a given call to the agent, needs 'None' as a valid type, since this is what this will start as
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add] # list of actions and corresponding observations, operations to this state should be added to the existing values

In [16]:
# define the nodes

from langchain_core.agents import AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor = ToolExecutor(tools)

def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {'agent_outcome': agent_outcome}

def execute_tools(data):
    # get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data['agent_outcome']
    output = tool_executor.invoke(agent_action)
    return {'intermediate_steps': [(agent_action, str(output))]}

def should_continue(data):
    # define logic that will be used to determine which conditional edge to go down
    if isinstance(data['agent_outcome'], AgentFinish):
        return 'end'

    # otherwise an AgentAction is returned
    # here we return `continue` string
    # this will be used when setting up the graph to define the flow
    else:
        return 'continue'

In [18]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(AgentState)

workflow.add_node('agent', run_agent)
workflow.add_node('action', execute_tools)

workflow.set_entry_point('agent')

workflow.add_conditional_edges(
    'agent',
    should_continue,
    {
        'continue': 'action',
        'end': END
    }
)

workflow.add_edge('action', 'agent')
app = workflow.compile()

In [19]:
inputs = {'input': 'what is the weather in SF', 'chat_history': []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print('----')

{'agent_outcome': AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call'}, id='run-6f0db9f4-df1f-4542-ac00-b6777d856866-0')])}
----
{'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call'}, id='run-6f0db9f4-df1f-4542-ac00-b6777d856866-0')])